In [1]:
import os
import gym
import ray
from animalai.envs.arena_config import ArenaConfig
from animalai.envs.gym.environment import AnimalAIGym
from ray.rllib.agents import ppo

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
from ray.tune import register_env, tune

In [3]:
class UnityEnvWrapper(gym.Env):
    def __init__(self, env_config):
        self.vector_index = env_config.vector_index
        self.worker_index = env_config.worker_index
        self.worker_id = env_config["unity_worker_id"] + env_config.worker_index
        self.env = AnimalAIGym(
            environment_filename = "/home/azibit/Downloads/AnimalAI-Olympics/examples/env/AnimalAI",
            worker_id = self.worker_id,
            flatten_branched = True,
            uint8_visual = True,
            arenas_configurations = ArenaConfig('/home/azibit/Downloads/AnimalAI-Olympics/examples/configurations/curriculum/0.yml')
        )
        self.action_space = self.env.action_space
        self.observation_space = self.env.observation_space
        
    def reset(self):
        return self.env.reset()
    
    def step(self, action):
        return self.env.step(action)

In [4]:
conf = {"env": "unity_env",
            "num_workers": 0,
            "env_config": {
                "unity_worker_id": 52
            },
        "num_gpus": int(os.environ.get("RLLIB_NUM_GPUS", "0")),
        "num_workers": 1,  # parallelism
        "framework": "torch",
        "train_batch_size": 500
       }
conf

{'env': 'unity_env',
 'num_workers': 1,
 'env_config': {'unity_worker_id': 52},
 'num_gpus': 0,
 'framework': 'torch',
 'train_batch_size': 500}

In [5]:
ray.shutdown()
ray.init()

register_env("unity_env", lambda config: UnityEnvWrapper(config))

2021-03-19 00:13:29,543	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8265


In [6]:
result = tune.run("PPO", stop = {"timesteps_total": 1000, "training_iteration": 3}, config = conf, checkpoint_at_end = True)

Trial name,status,loc
PPO_unity_env_d9a69_00000,RUNNING,


(pid=34746) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=34746) Instructions for updating:
(pid=34746) non-resource variables are not supported in the long term
(pid=34746) 2021-03-19 00:13:34,927	INFO trainer.py:643 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=34745) WARNING:tensorflow:From /home/azibit/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/compat/v2_compat.py:68: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=34745) Instructions for updating:
(pid=34745) non-resource variables are not supported in the long term


(pid=34745) Found path: /home/azibit/Downloads/AnimalAI-Olympics/examples/env/AnimalAI.x86_64
(pid=34745) Mono path[0] = '/home/azibit/Downloads/AnimalAI-Olympics/examples/env/AnimalAI_Data/Managed'
(pid=34745) Mono config path = '/home/azibit/Downloads/AnimalAI-Olympics/examples/env/AnimalAI_Data/MonoBleedingEdge/etc'
(pid=34745) Preloaded 'lib_burst_generated.so'
(pid=34745) Preloaded 'libgrpc_csharp_ext.x64.so'
(pid=34745) Preloaded 'ScreenSelector.so'
(pid=34745) Display 0 '0': 1920x1080 (primary device).
(pid=34745) Logging to /home/azibit/.config/unity3d/Unity Technologies/UnityEnvironment/Player.log


(pid=34745) INFO:mlagents_envs:Connected to Unity environment with package version 0.15.0-preview and communication version 0.15.0
(pid=34745) INFO:mlagents_envs:Connected new brain:
(pid=34745) AnimalAI?team=0
(pid=34745) INFO:gym_unity:1 agents within environment.
(pid=34745) 2021-03-19 00:13:40,296	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
(pid=34746) 2021-03-19 00:13:40,440	WARNING deprecation.py:34 -- DeprecationWarning: `framestack` has been deprecated. Use `num_framestacks (int)` instead. This will raise an error in the future!
(pid=34746) 2021-03-19 00:13:40,594	WARNING util.py:47 -- Install gputil for GPU system monitoring.
(pid=34745) 2021-03-19 00:13:40,618	WARNING deprecation.py:34 -- DeprecationWarning: `env_index` has been deprecated. Use `episode.env_id` instead. This will raise an error in the future!


Result for PPO_unity_env_d9a69_00000:
  custom_metrics: {}
  date: 2021-03-19_00-14-31
  done: false
  episode_len_mean: 9.966666666666667
  episode_reward_max: 0.975000006146729
  episode_reward_mean: 0.9602888968152304
  episode_reward_min: 0.88033336866647
  episodes_this_iter: 60
  episodes_total: 60
  experiment_id: 41340d0296b1412cb96e4f2aa1b78dec
  hostname: azibit-Lenovo-Y520-15IKBM
  info:
    learner:
      default_policy:
        allreduce_latency: 0.0
        cur_kl_coeff: 0.2
        cur_lr: 5.0e-05
        entropy: 0.3387770354747772
        entropy_coeff: 0.0
        kl: 0.1652733027935028
        policy_loss: 0.041499136760830876
        total_loss: 1.3208546400070191
        vf_explained_var: -0.20183400809764862
        vf_loss: 1.2463008522987367
    num_steps_sampled: 600
    num_steps_trained: 600
  iterations_since_restore: 1
  node_ip: 127.0.1.1
  num_healthy_workers: 1
  off_policy_estimator: {}
  perf:
    cpu_util_percent: 24.556164383561647
    ram_util_perce

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_unity_env_d9a69_00000,RUNNING,127.0.1.1:34746,1,50.683,600,0.960289,0.975,0.880333,9.96667


Result for PPO_unity_env_d9a69_00000:
  custom_metrics: {}
  date: 2021-03-19_00-15-28
  done: true
  episode_len_mean: 11.57
  episode_reward_max: 0.975000006146729
  episode_reward_mean: 0.9539866749290377
  episode_reward_min: 0.8056666702032089
  episodes_this_iter: 44
  episodes_total: 104
  experiment_id: 41340d0296b1412cb96e4f2aa1b78dec
  hostname: azibit-Lenovo-Y520-15IKBM
  info:
    learner:
      default_policy:
        allreduce_latency: 0.0
        cur_kl_coeff: 0.30000000000000004
        cur_lr: 5.0e-05
        entropy: 0.35833298563957217
        entropy_coeff: 0.0
        kl: 0.032970056682825086
        policy_loss: -0.016580140963196755
        total_loss: 0.017722992599010466
        vf_explained_var: -0.19906166195869446
        vf_loss: 0.024412114918231965
    num_steps_sampled: 1200
    num_steps_trained: 1200
  iterations_since_restore: 2
  node_ip: 127.0.1.1
  num_healthy_workers: 1
  off_policy_estimator: {}
  perf:
    cpu_util_percent: 33.56463414634146
   

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_unity_env_d9a69_00000,RUNNING,127.0.1.1:34746,2,107.911,1200,0.953987,0.975,0.805667,11.57


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_unity_env_d9a69_00000,TERMINATED,,2,107.911,1200,0.953987,0.975,0.805667,11.57


2021-03-19 00:15:29,236	INFO tune.py:450 -- Total run time: 116.34 seconds (115.64 seconds for the tuning loop).
(pid=34745) INFO:mlagents_envs:Environment shut down with return code 0.


In [16]:
result.results

{'d9a69_00000': {'episode_reward_max': 0.975000006146729,
  'episode_reward_min': 0.8056666702032089,
  'episode_reward_mean': 0.9539866749290377,
  'episode_len_mean': 11.57,
  'episodes_this_iter': 44,
  'policy_reward_min': {},
  'policy_reward_max': {},
  'policy_reward_mean': {},
  'custom_metrics': {},
  'hist_stats': {'episode_reward': [0.9643333628773689,
    0.9536666600033641,
    0.9576666597276926,
    0.9683333626016974,
    0.8656666660681367,
    0.9683333626016974,
    0.9656666591763496,
    0.9683333626016974,
    0.9683333626016974,
    0.9683333626016974,
    0.9656666591763496,
    0.9470000080764294,
    0.9616666594520211,
    0.9510000078007579,
    0.9150000102818012,
    0.9696666589006782,
    0.975000006146729,
    0.9616666594520211,
    0.9736666586250067,
    0.9696666589006782,
    0.975000006146729,
    0.9683333626016974,
    0.8496666671708226,
    0.9683333626016974,
    0.975000006146729,
    0.8056666702032089,
    0.9616666594520211,
    0.9683333

In [ ]:
from ray.rllib.agents.ppo import PPOTrainer

ray.shutdown()
ray.init()

register_env("unity_env", lambda config: UnityEnvWrapper(config))
res = tune.run(PPOTrainer, config=conf)